In [37]:
import os
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import PyPDF2
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains import create_history_aware_retriever

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [ ]:
load_dotenv()
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
HF_KEY=os.getenv("HF_KEY")

llm=ChatGroq(model="Llama3-8b-8192",groq_api_key=GROQ_API_KEY)
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

g:\ANN,RNN,LSTM proj\ANN\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#reading pdf file

file_path="G:\Resume projects\RAG-Based Question Answering System\std4-bb-evs2.pdf".strip()

with open(file_path,'rb') as file:
    reader=PyPDF2.PdfReader(file)
    text="\n".join([page.extract_text() for page in reader.pages])

In [23]:
text

'\nMaharashtra State Bureau of Textbook Production\nand Curriculum Research, Pune.Education Department’s Sanction Number :\nPra Shi Sa  / 2014  -15 / 2102  / Manjuri  / D - 505 / 755 Dated 4.2.2014\nSHIVACHHATRAPATI\n(ENVIRONMENTAL STUDIES - PART TWO)\nSTANDARD FOUR\n\nMaharashtra State Bureau of Textbook Production and Curriculum Research, Pune - 411 004.\nThe Maharashtra State Bureau of Textbook Production and Curriculum Research \nreserves all the rights relating to the book. No part of this  book  should be reproduced without the  written permission of  the Director, Maharashtra State Bureau of Textbook Production and Curriculum Research, Pune.\nPreface\n The ‘Primary Education Curriculum - 2012’ was prepared \nin the State of Maharashtra following the ‘Right of Children to Free and Compulsory Education Act, 2009’ and the ‘National Curriculum Framework 2005’.  The syllabus is being \nimplemented serially from the academic year 2013-2014.  \n The syllabus includes the subjects Gener

In [25]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

splitted_text=splitter.split_text(text)

In [29]:
splitted_text[:5]

['Maharashtra State Bureau of Textbook Production\nand Curriculum Research, Pune.Education Department’s Sanction Number :\nPra Shi Sa  / 2014  -15 / 2102  / Manjuri  / D - 505 / 755 Dated 4.2.2014\nSHIVACHHATRAPATI\n(ENVIRONMENTAL STUDIES - PART TWO)\nSTANDARD FOUR',
 'Maharashtra State Bureau of Textbook Production and Curriculum Research, Pune - 411 004.\nThe Maharashtra State Bureau of Textbook Production and Curriculum Research \nreserves all the rights relating to the book. No part of this  book  should be reproduced without the  written permission of  the Director, Maharashtra State Bureau of Textbook Production and Curriculum Research, Pune.\nPreface\n The ‘Primary Education Curriculum - 2012’ was prepared \nin the State of Maharashtra following the ‘Right of Children to Free and Compulsory Education Act, 2009’ and the ‘National Curriculum Framework 2005’.  The syllabus is being \nimplemented serially from the academic year 2013-2014.  \n The syllabus includes the subjects Gener

In [47]:
vectore_store=FAISS.from_texts(splitted_text,embedding=embedding)

retriver=vectore_store.as_retriever()

In [48]:
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [49]:
contextualize_q_system_prompt=(
            "Given a chat history and the latest user question"
            "which might reference context in the chat history, "
            "formulate a standalone question which can be understood "
            "without the chat history. Do NOT answer the question, "
            "just reformulate it if needed and otherwise return it as is."
        )
contextualize_q_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", contextualize_q_system_prompt),
                    MessagesPlaceholder("chat_history"),
                    ("human", "{input}"),
                ]
            )

In [50]:
history_aware_retriever=create_history_aware_retriever(llm,retriver,contextualize_q_prompt)

In [51]:
system_prompt="""
    You are a great historian having knowledge of indian history,
    given a question try to answer it in depth by refering given
    context from retriever.
    <context>
    {context}
    <context>
"""

prompt=ChatPromptTemplate([
    ("system",system_prompt),
    MessagesPlaceholder("chat_history"),
    ("user","{input}")
])

In [52]:
qa_prompt=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,qa_prompt)

In [53]:
conversational_rag_chain=RunnableWithMessageHistory(
    rag_chain,get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [54]:
configs={"configurable":{"session_id":"id1"}}
response=conversational_rag_chain.invoke({"input":"Tell me about maharaj in nutshell"},config=configs)

response

{'input': 'Tell me about maharaj in nutshell',
 'chat_history': [],
 'context': [Document(metadata={}, page_content='After the conclusion of this successful \ncampaign of Karnatak Shivaji returned to Raigad. While he was yet to overcome the strain of this campaign, he had to undertake a naval campaign against the  Siddi of Janjira. At this time, Maharaj was \nfifty years old. He had toiled continuously  for nearly thirty-five years, without any \ntime to rest.\nThe People’s Protector Is No More  : \nOn 3rd April, 1680, Shivaji Maharaj died, \nplunging his people into a sea of sorrow. The protector of people passed away.\nIn his lifetime, Maharaj achieved so \nmany great things. He vanquished very powerful enemies and founded Hindavi Swaraj. There was none in the whole of \nIndia who could match his achievements. \nHe was a great national figure.\nlllShivaji Maharaj meets Vyankoji Raje.'),
  Document(metadata={}, page_content='women. He had strictly warned his \nsoldiers not to drink al

In [55]:
response=conversational_rag_chain.invoke({"input":"what was my last question"},config=configs)

response

{'input': 'what was my last question',
 'chat_history': [HumanMessage(content='Tell me about maharaj in nutshell', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Shivaji Maharaj was a great Indian king and warrior who founded the Maratha Empire in the 17th century. He is considered one of the greatest figures in Indian history. Here is a brief summary of his life and achievements:\n\n* Born: 1627\n* Died: 1680\n* Age at death: 53\n* Established the Maratha Empire in 1674\n* Fought against the Mughal Empire and the Adilshahi Sultanate\n* Founded Hindavi Swaraj, a government based on justice, fair play, and equal treatment to people of all castes and religions\n* Unified the fragmented Maratha territories and established a strong administration\n* Fought against the Siddis of Janjira and conquered many forts, including Sindhudurg and Vijaydurg\n* Established a strong navy and is considered the "Father of the Indian Navy"\n* Was a great patron of education, arts, and cu

In [56]:
response=conversational_rag_chain.invoke({"input":"what is the birth place of maharaj"},config=configs)

response

{'input': 'what is the birth place of maharaj',
 'chat_history': [HumanMessage(content='Tell me about maharaj in nutshell', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Shivaji Maharaj was a great Indian king and warrior who founded the Maratha Empire in the 17th century. He is considered one of the greatest figures in Indian history. Here is a brief summary of his life and achievements:\n\n* Born: 1627\n* Died: 1680\n* Age at death: 53\n* Established the Maratha Empire in 1674\n* Fought against the Mughal Empire and the Adilshahi Sultanate\n* Founded Hindavi Swaraj, a government based on justice, fair play, and equal treatment to people of all castes and religions\n* Unified the fragmented Maratha territories and established a strong administration\n* Fought against the Siddis of Janjira and conquered many forts, including Sindhudurg and Vijaydurg\n* Established a strong navy and is considered the "Father of the Indian Navy"\n* Was a great patron of education, art

In [57]:
response=conversational_rag_chain.invoke({"input":"can u tell me history of that fort"},config=configs)

response

{'input': 'can u tell me history of that fort',
 'chat_history': [HumanMessage(content='Tell me about maharaj in nutshell', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Shivaji Maharaj was a great Indian king and warrior who founded the Maratha Empire in the 17th century. He is considered one of the greatest figures in Indian history. Here is a brief summary of his life and achievements:\n\n* Born: 1627\n* Died: 1680\n* Age at death: 53\n* Established the Maratha Empire in 1674\n* Fought against the Mughal Empire and the Adilshahi Sultanate\n* Founded Hindavi Swaraj, a government based on justice, fair play, and equal treatment to people of all castes and religions\n* Unified the fragmented Maratha territories and established a strong administration\n* Fought against the Siddis of Janjira and conquered many forts, including Sindhudurg and Vijaydurg\n* Established a strong navy and is considered the "Father of the Indian Navy"\n* Was a great patron of education, art